In [27]:
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as lite
import sys
from pyne import nucname
from pyne.material import Material

/Users/13b/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/13b/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


In [28]:
# get sqlite cursor
filename = '../../db/1yr.sqlite'
con = lite.connect(filename)
# allows indexing with column name
con.row_factory = lite.Row
cur = con.cursor()

# Finding the Average Assembly and its Composition 

In [33]:
# first, find the average composition and recipe 
average_burnup = cur.execute('SELECT avg(discharge_burnup) FROM discharge').fetchone()[0]
average_enrichment = cur.execute('SELECT avg(initial_enrichment) FROM discharge').fetchone()[0]
print('AVG BURNUP: %f MWD/MTHM \nAVG ENRICHMENT: %f wt%% U235' %(average_burnup, average_enrichment))

AVG BURNUP: 36169.381303 MWD/MTHM 
AVG ENRICHMENT: 3.392045 wt% U235


In [34]:
# find assembly closest to this value:
min_diff_assem_id = cur.execute('SELECT assembly_id, min(abs(discharge_burnup - %f) + abs(initial_enrichment - %f)* 10000) '
                                'FROM discharge' %(average_burnup, average_enrichment)).fetchone()[0]
print(min_diff_assem_id)

324192


In [35]:
# how different is this assembly from the average?
chosen_assem = cur.execute('SELECT * FROM discharge WHERE assembly_id = %i' %min_diff_assem_id).fetchall()
burnup = chosen_assem[0]['discharge_burnup']
enrichment = chosen_assem[0]['initial_enrichment']

# print error with the average value:
print('ERROR:')
enr_err = 100 * (average_enrichment - enrichment) / average_enrichment
bur_err = 100 * (average_burnup - burnup) / average_burnup
print('ENRICHMENT: %f %% \nBURNUP %f %%' %(enr_err, bur_err))

ERROR:
ENRICHMENT: -0.057637 % 
BURNUP -0.007240 %


In [ ]:
# save this assembly's composition to a dictionary
recipe = {}
total_mass = 0
for row in chosen_assem:
    isotope = nucname.name(row['isotope'])
    mass = float(row['total_mass_g'])
    total_mass += mass
    recipe[isotope] = mass

# normalize values:
for iso in recipe:
    recipe[iso] = recipe[iso] / total_mass

print(recipe)

# We store all the assemblies in a dictionary, one with the composition with UNF-ST&DARDS, and the other with recipe composition

In [38]:
def get_assembly_evaluation_times(cur):
    """ gets assembly evaluation times. """
    assem_dict = {}
    id_and_date = cur.execute('SELECT distinct(assembly_id), evaluation_date FROM discharge').fetchall()
    for row in id_and_date:
        assem_dict[row['assembly_id']] = {'date': row['evaluation_date']}
    return assem_dict

In [24]:
def attach_comp_mass(cur, assem_dict):
    for key, value in assem_dict.items():
        assem_id = key
        comp_query = cur.execute('SELECT isotope, total_mass_g FROM discharge '
                                 'WHERE assembly_id = %s' %assem_id).fetchall()
        total_mass = 0
        # initialize key comp
        assem_dict[key]['comp'] = {}
        
        # save composition into comp key
        for row in comp_query:
            nuclide_name = nucname.name(row['isotope'])
            mass = float(row['total_mass_g']) 
            total_mass += mass
            assem_dict[key]['comp'][nuclide_name] = mass
        # add mass key / val
        assem_dict[key]['mass'] = total_mass
        # normalize values
        for iso in assem_dict[key]['comp']:
            assem_dict[key]['comp'][iso] = assem_dict[key]['comp'][iso] / total_mass
    return assem_dict

In [45]:
def get_assembly_dict(cur):
    """ gets assembly evalution times, isotope, total_mass_g and total mass"""
    assem_dict = {}
    query = cur.execute('SELECT * FROM discharge').fetchall()
    count = 0
    percent_done = 0
    for row in query:
        iso = nucname.name(row['isotope'])
        mass = float(row['total_mass_g'])
        if row['assembly_id'] not in assem_dict.keys():
            # first of the assembly input data, initialize assem_dict[assembly_id] dict
            assem_dict[row['assembly_id']] = {'date': row['evaluation_date']}
            assem_dict[row['assembly_id']]['mass'] = 0
            assem_dict[row['assembly_id']]['comp'] = {iso: mass}
        else:
            assem_dict[row['assembly_id']]['comp'][iso] = mass
            assem_dict[row['assembly_id']]['mass'] += mass
        count += 1
        if len(query) // 50 == count:
            percent_done += 2
            print('%s %% DONE' %percent_done)
    return assem_dict

In [46]:
def convert_to_recipe(assem_dict, recipe):
    recipe_dict = assem_dict.copy()
    if sum(recipe.values()) < 0.99 or sum(recipe.values()) > 1.001:
        raise ValueError('This aint normalized son')
    for key, value in recipe_dict.items():
        recipe_dict[key]['comp'] = recipe
    return recipe_dict

In [ ]:
# this takes a while:
assem_dict = get_assembly_dict(cur)

In [ ]:
recipe_dict = convert_to_recipe(assem_dict)

# Then we convert the assembly to `pyne` material, for decay and analysis

In [ ]:
def attach_pyne_material(assem_dict):
    for key, value in assem_dict.items():
        pyne_mat = Material(value['comp'], value['mass'])
        assem_dict[key]['mat'] = pyne_mat

In [ ]:
assem_dict = attach_pyne_material(assem_dict)
recipe_dict = attach_pyne_material(recipe_dict)

# Then we decay the assemblies to 2020:

In [ ]:
def find_diff_time_secs(year, month, day, evaluation_date):
    # the UNF-ST&DARDS data format is YYYY-MM-DD
    ev_year = int(evaluation_date[:4])
    ev_month = int(evaluation_date[5:7])
    ev_day = int(evaluation_date[8:])
    
    dyear = 0
    dmonth = 0
    dday = day - ev_day
    if dday < 0:
        dmonth -= 1
        dday += 30
    dmonth += month - ev_month
    if dmonth < 0:
        dyear -= 1
        dmonth += 12
    dyear += year - ev_year
    if dyear < 0:
        raise ValueError('Cannot go back in time man')
    
    time_in_sec = dyear * (365 * 30 * 24 * 3600) + dmonth * (30 * 24 * 3600) + dday * (25 * 3600)
    return time_in_sec

In [ ]:
def decay_assemblies(assem_dict):
    decayed_dict = assem_dict.copy()
    for key, value in assem_dict.items():
        # to 2020-07-01
        decay_time = find_diff_time_secs(2020, 7, 1, assem_dict[key]['date'])
        assem_dict[key]['mat'] = assem_dict[key]['mat'].decay(decay_time)
    return assem_dict

In [ ]:
2020_assem_dict = decay_assemblies(assem_dict)
2020_recipe_dict = decay_assemblies(recipe_dict)

# For easier analysis, we collapse all the assemblies into one giant `pyne` material

In [ ]:
def combine_material(assem_dict):
    collapsed_mat = Material({'H1':1}, 1e-6)
    for key, value in assem_dict.items():
        decayed_material = assem_dict[key]['mat']
        collapsed_mat = collapsed_mat + decayed_material
    return collapsed_mat

# Then we compare the metrics:

In [ ]:
def decay_heat(assem_dict):
    